In [16]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# updating nipype from 1.7.0 -> 1.8.1, will hopefully ifx no PE estimates founds error # didn't fix

### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

In [17]:
# dataset = 'Leipzig_7T_SM'
dataset = 'Leipzig_7T_GdH'

#### !!! must manually change subject numbers in nipype code based on dataset !!!

In [18]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

if dataset == 'Leipzig_7T_SM':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['1p6']
    
elif dataset == 'Leipzig_7T_GdH':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['5p0'] #['3p125'] # ['1p5']
    
if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model

subject_ids = [x.split('/')[-5].split('-')[-1] for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 
                                     'subject_level_model', dataset, 'sub-*', 'func',f'fwhm-{fwhms[0]}',f'model-{model_ns[0]}',
                                     f'*task-{task}_space-MNI*contrast-0_desc-zstat*')))]
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', dataset)
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', dataset)

template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "group_level_model", dataset,f'model-{model_ns[0]}')

templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stop_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stop_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stop_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}


In [19]:
print(f"""
dataset : {dataset}
fwhm : {fwhms}
model_n : {model_ns}
task : {task}
no. subs : {len(subject_ids)}
subs : {subject_ids}
work_dir : {work_dir}
slm folder : {slm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level2_cope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level2_varcope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level2_tdof'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
""")


dataset : Leipzig_7T_GdH
fwhm : ['5p0']
model_n : ['0']
task : stop
no. subs : 17
subs : ['BI3T', 'FMFT', 'GAIT', 'KCAT', 'KP6T', 'LV2T', 'MRCT', 'NM3T', 'PF5T', 'RSIT', 'SC1T', 'SPGT', 'TS6T', 'UM2T', 'VL1T', 'WW2T', 'ZK4T']
work_dir : /home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH
slm folder : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/subject_level_model/Leipzig_7T_GdH
output dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0
contrasts : ['0', '1', '2', '3', '4', '5']

## if uneven number of template / designs / subs the code will crash
no.copes = 17
no.varvopes = 17
no.tdofs = 17



In [20]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [21]:
##
workflow = pe.Workflow(name='feat_level3_sst_hp')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [22]:
def get_subject_ids():

    subject_ids = ['BI3T', 'FMFT', 'GAIT', 'KCAT', 'KP6T', 'LV2T', 'MRCT', 'NM3T', 
                   'PF5T', 'RSIT', 'SC1T', 'SPGT', 'TS6T', 'UM2T', 'VL1T', 'WW2T', 'ZK4T']
        
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids): #=subject_ids):
    print(second_level_contrast)
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors


contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')



In [23]:
# flameo = pe.Node(
#     interface=fsl.FLAMEO(run_mode='flame1'),
#     name="flame1")

# flameo.inputs.mask_file = template_brain_mask

# workflow.connect([
#     (copemerge, flameo, [('merged_file', 'cope_file')]),
#     (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
#     (level3model, flameo, [('design_mat', 'design_file'),
#                            ('design_con', 't_con_file'), 
#                            ('design_grp', 'cov_split_file')]),
# ])

flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [24]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [25]:
# ## Datasink
# datasink = pe.Node(nio.DataSink(), name='sinker')
# datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-sstmsit","task-sst")

# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

# ## todo: substitutions
# workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
# workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
# workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
# workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

# ## cluster results
# workflow.connect(grf_cluster, 'threshold_file', datasink, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'grf_localmax_txt_file')
# workflow.connect(grf_cluster, 'index_file', datasink, 'cluster_indices')

In [26]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir

if len(fwhms[0]) == 3:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]
elif len(fwhms[0]) == 5:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]    

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [27]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 100})

221003-10:09:59,376 nipype.workflow INFO:
	 Workflow feat_level3_sst_hp settings: ['check', 'execution', 'logging', 'monitoring']
221003-10:09:59,709 nipype.workflow INFO:
	 Running in parallel.
221003-10:09:59,713 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 100.00/100.00, Free processors: 10/10.
221003-10:09:59,773 nipype.workflow INFO:
	 [Job 0] Cached (feat_level3_sst_hp.subject_id_getter).
221003-10:09:59,775 nipype.workflow INFO:
	 [Job 1] Cached (feat_level3_sst_hp.get_volume).
221003-10:09:59,812 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH/feat_level3_sst_hp/_contrast_n_1_fwhm_5p0_model_n_0/selector".
221003-10:09:59,813 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH/feat_level3_sst_hp/_contr

221003-10:10:02,154 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l3", elapsed time 0.002219s.
221003-10:10:03,716 nipype.workflow INFO:
	 [Job 8] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:03,718 nipype.workflow INFO:
	 [Job 9] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:03,719 nipype.workflow INFO:
	 [Job 10] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:03,720 nipype.workflow INFO:
	 [Job 11] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:03,723 nipype.workflow INFO:
	 [Job 12] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:03,725 nipype.workflow INFO:
	 [Job 13] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:03,727 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 14 jobs ready. Free memory (GB): 99.20/100.00, Free processors: 6/10.
                     Currently running:
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
  

221003-10:10:25,741 nipype.workflow INFO:
	 [Job 21] Completed (feat_level3_sst_hp.varcopemerge).
221003-10:10:25,742 nipype.workflow INFO:
	 [Job 22] Completed (feat_level3_sst_hp.copemerge).
221003-10:10:25,743 nipype.workflow INFO:
	 [Job 23] Completed (feat_level3_sst_hp.varcopemerge).
221003-10:10:25,744 nipype.workflow INFO:
	 [Job 26] Completed (feat_level3_sst_hp.l3model).
221003-10:10:25,746 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 7 jobs ready. Free memory (GB): 99.60/100.00, Free processors: 8/10.
                     Currently running:
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
221003-10:10:25,794 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.l3model" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH/feat_level3_sst_hp/_contrast_n_1_fwhm_5p0_model_n_0/l3model".
221003-10:10:25,795 nipype.workflow INFO:
	 [Node] Setting-up "feat_l

221003-10:10:41,815 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.flameo" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH/feat_level3_sst_hp/_contrast_n_4_fwhm_5p0_model_n_0/_run_mode_flame1/flameo".
221003-10:10:41,912 nipype.workflow INFO:
	 [Node] Executing "flameo" <nipype.interfaces.fsl.model.FLAMEO>
221003-10:10:43,609 nipype.workflow INFO:
	 [Node] Finished "varcopemerge", elapsed time 19.754489s.
221003-10:10:43,753 nipype.workflow INFO:
	 [Job 25] Completed (feat_level3_sst_hp.varcopemerge).
221003-10:10:43,756 nipype.workflow INFO:
	 [MultiProc] Running 9 tasks, and 3 jobs ready. Free memory (GB): 98.20/100.00, Free processors: 1/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_ss

221003-10:15:10,253 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH/feat_level3_sst_hp/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame1/smoothestimate/mapflow/_smoothestimate1".
221003-10:15:10,358 nipype.workflow INFO:
	 [Node] Executing "_smoothestimate1" <nipype.interfaces.fsl.model.SmoothEstimate>
221003-10:15:11,434 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate1", elapsed time 1.064428s.
221003-10:15:12,25 nipype.workflow INFO:
	 [Job 104] Completed (_smoothestimate0).
221003-10:15:12,28 nipype.workflow INFO:
	 [Job 105] Completed (_smoothestimate1).
221003-10:15:12,31 nipype.workflow INFO:
	 [MultiProc] Running 8 tasks, and 7 jobs ready. Free memory (GB): 98.40/100.00, Free processors: 2/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat

...

221003-15:32:54,466 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
221003-15:32:54,468 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>
221003-15:32:54,471 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
221003-15:32:56,132 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 1.662995s.
221003-15:32:56,246 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 1.7737910000000001s.
221003-15:32:56,382 nipype.workflow INFO:
	 [Job 167] Completed (_grf_cluster1).
221003-15:32:56,383 nipype.workflow INFO:
	 [Job 169] Completed (_grf_cluster1).
221003-15:32:56,385 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 0 jobs ready. Free memory (GB): 99.40/100.00, Free processors: 7/10.
                     Currently running:
                       * _grf_cluster0
                       * _grf_cluster0
                 

221003-15:33:00,532 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/third_level_model/level3_varcopes/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/varcope2.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-0_grouplevelcontrast-2_flame-12_desc-varcope.nii.gz
221003-15:33:00,533 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/third_level_model/level3_tdof_ts/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/tdof_t1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-0_grouplevelcontrast-1_flame-12_desc-tdof_t.nii.gz
221003-15:33:00,534 nipype.interface INFO:
	 sub: /

221003-15:34:55,261 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate0", elapsed time 0.703171s.
221003-15:34:55,296 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate1", elapsed time 0.737433s.
221003-15:34:56,493 nipype.workflow INFO:
	 [Job 170] Completed (_smoothestimate0).
221003-15:34:56,494 nipype.workflow INFO:
	 [Job 171] Completed (_smoothestimate1).
221003-15:34:56,496 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 100.00/100.00, Free processors: 10/10.
221003-15:34:56,553 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_GdH/feat_level3_sst_hp/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/smoothestimate/mapflow/_smoothestimate0".
221003-15:34:56,555 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate0" - collecting precomputed outputs
221003-15:34:56,555 nipype.workflow INFO:
	 [Node] "_smoothestimate0" found 

221003-15:35:06,614 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/third_level_model/level3_zstats/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/zstat1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-1_flame-12_desc-zstat.nii.gz
221003-15:35:06,615 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_2.3/_grf_cluster1/zstat2_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-2_flame-12_desc-zstat_voxelthreshold

221003-15:35:06,624 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_3.1/_grf_cluster1/zstat2_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_GdH/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-2_flame-12_desc-zstat_voxelthreshold-3.1.nii.gz
221003-15:35:06,624 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.011479s.
221003-15:35:06,626 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.012741s.
221003-15:35:08,505 nipype.workflow INFO:
	 [Job 90] Completed (feat_level3_sst_hp.sinker).
221003-15:35:08,507 nipype.workflow INFO:
	 [Job 91] Completed (feat_level3_sst_hp.sinker).
221003-15:35:08,509 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs re